# Resnet Classification to ImageNet Categories

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.data import get_image_from_url
from utils.data import init_model_logging
import matplotlib.pyplot as plt
import numpy as np
import pickle
from tensorflow.contrib.slim.nets import resnet_v1
from tensorflow.contrib import slim
import tensorflow as tf

## Load Model Meta

 - http://www.image-net.org/search?q=dog

In [ ]:
resnet_model_ckpt = '/data/checkpoints/resnet_v1_50.ckpt'
renset_class_names = '/data/checkpoints/resnet_v1_50_catnames.pickle'

with open(renset_class_names, 'rb') as fr:
    imagenet_category_names = pickle.load(fr)

In [ ]:
imagenet_category_names

## Build Resnet Graph

 - https://github.com/tensorflow/models/tree/master/research/slim 
 - https://github.com/tensorflow/models/blob/master/research/slim/nets/resnet_v1.py   

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.variable_scope('resnet_inputs'):
        images = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
        is_training = tf.placeholder(tf.bool)
        
    with slim.arg_scope(resnet_v1.resnet_arg_scope()):
        raw_prediction, othere_layers = resnet_v1.resnet_v1_50(
            images, output_stride=None, num_classes=1000, global_pool=True, 
            is_training=is_training, scope='resnet_v1_50')
        
    with tf.name_scope('softmax_prediction'):
        prediction = tf.nn.softmax(tf.reshape(raw_prediction, shape=(-1, 1000)))
    
    init_resnet = slim.assign_from_checkpoint_fn(resnet_model_ckpt, slim.get_model_variables('resnet_v1_50'))
    initialize_vars = tf.group(
        tf.global_variables_initializer(),
        tf.local_variables_initializer())

## Init Model Logging

In [ ]:
base_dir = '/tensorboard_summaries/resnet/'
logging_meta = init_model_logging(base_dir, 'experiment_1', graph=graph, remove_existing=True)

## Load Image

In [ ]:
url = 'http://british-samoyed-club.co.uk/bsc/wp-content/uploads/scooter.jpg'

In [ ]:
# Download image and reshape to batch format.
image_shape = [224, 224, 3]
samoyed_img = get_image_from_url(url, image_shape)
samoyed_img = samoyed_img.reshape([-1, 224, 224, 3])

# Plot image.
plt.figure(figsize=(10, 10))
plt.imshow(samoyed_img[0].astype(np.uint8))
ax = plt.gca()
ax.axes.set_axis_off()

## Run Net

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
        
with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    init_resnet(session)
    _prediction, = session.run([prediction], feed_dict={is_training: False, images: samoyed_img})

## Results

In [ ]:
output_softmax = _prediction[0]
top_n = 10
top_label_class_ids = np.argsort(output_softmax)[-1:0:-1][:top_n]
top_label_class_scores = output_softmax[top_label_class_ids]

figure = plt.figure(figsize=(10, 10))
ax = plt.gca()
rects = ax.bar(np.arange(top_n), top_label_class_scores, 0.5, color='r')

ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
results = plt.xticks(np.arange(top_n), [imagenet_category_names[cat_id] for cat_id in top_label_class_ids], rotation='vertical')

## Corrupt Image with Patch

### Load patches

In [ ]:
url = "https://pbs.twimg.com/media/DSU7iNMU8AAciHy.png"

In [ ]:
# Donwload patches.
image_shape = [224, 224, 3]
patches_img = get_image_from_url(url, image_shape)
patches_img = patches_img.reshape([-1, 224, 224, 3])

# Plot patches.
plt.figure(figsize=(10, 10))
plt.imshow(patches_img[0].astype(np.uint8))
ax = plt.gca()
ax.axes.set_axis_off()

In [ ]:
# Pick one of patches.
row_id = 1
col_id = 0
toaster_patch_img = patches_img[0][row_id*112:(row_id+1)*112, col_id*112:(col_id+1)*112, :]

# Plot patch.
plt.figure(figsize=(10, 10))
plt.imshow(toaster_patch_img.astype(np.uint8))
ax = plt.gca()
ax.axes.set_axis_off()

### Corrupt image

In [ ]:
# Merge patch with original image.
toaster_pixel_ids = np.where(toaster_patch_img > 0)

corruptd_samoyed_img = samoyed_img.copy()
corruptd_samoyed_img[0][toaster_pixel_ids] = toaster_patch_img[toaster_pixel_ids]

# Plot Merge.
plt.figure(figsize=(10, 10))
plt.imshow(corruptd_samoyed_img[0].astype(np.uint8))
ax = plt.gca()
ax.axes.set_axis_off()

### Run Net and Plot Results

In [ ]:
# Run net on corrupted image.
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
        
with tf.Session(graph=graph, config=config) as session:
    session.run([initialize_vars])
    init_resnet(session)
    _prediction, = session.run([prediction], feed_dict={is_training: False, images: corruptd_samoyed_img})
    
# Plot results.
output_softmax = _prediction[0]
top_n = 10
top_label_class_ids = np.argsort(output_softmax)[-1:0:-1][:top_n]
top_label_class_scores = output_softmax[top_label_class_ids]

figure = plt.figure(figsize=(10, 10))
ax = plt.gca()
rects = ax.bar(np.arange(top_n), top_label_class_scores, 0.5, color='r')

ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
results = plt.xticks(np.arange(top_n), [imagenet_category_names[cat_id] for cat_id in top_label_class_ids], rotation='vertical')

## Examine Strange Images

### Load Strange Images

In [ ]:
url = 'http://www.evolvingai.org/files/70_images_entry_v2_web.jpg'

In [ ]:
# Load strange images with appropriate size.
image_shape = [7 * 330, 10 * 224, 3]
strange_imgs = get_image_from_url(url, image_shape)
strange_imgs = strange_imgs.reshape([-1, 7 * 330, 10 * 224, 3])

# Plot images.
plt.figure(figsize=(20, 20))
plt.imshow(strange_imgs[0].astype(np.uint8))
ax = plt.gca()
ax.axes.set_axis_off()

### Pick One Strange Image

In [ ]:
row_id = 6
col_id = 4
strange_img = strange_imgs[:, row_id*330: row_id*330 + 224, col_id*224:(col_id+1)*224,:]

plt.figure(figsize=(20, 20))
plt.imshow(strange_img[0].astype(np.uint8))
ax = plt.gca()
ax.axes.set_axis_off()

### Run Net and Plot Results

In [ ]:
# Run net on strange image.
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
        
with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    init_resnet(session)
    _prediction, = session.run([prediction], feed_dict={is_training: False, images: strange_img})

# Plot results.  
output_softmax = _prediction[0]
top_n = 10
top_label_class_ids = np.argsort(output_softmax)[-1:0:-1][:top_n]
top_label_class_scores = output_softmax[top_label_class_ids]

figure = plt.figure(figsize=(10, 10))
ax = plt.gca()
rects = ax.bar(np.arange(top_n), top_label_class_scores, 0.5, color='r')

ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
results = plt.xticks(np.arange(top_n), [imagenet_category_names[cat_id] for cat_id in top_label_class_ids], rotation='vertical')